In [1]:
import pandas as pd

In [2]:
matches = pd.read_csv("matches.csv", index_col=0)

In [3]:
matches.head()

,Data,Ora,Competizione,Girone,Giorno,Stadio,Risultato,Rf,Rs,Avversario,...,Arbitro,Report partita,Note,Tiri,Dist.,Pun.,Rigori,Rig T,Stagione,Squadra
0,19-08-2023,20:45,Serie A,Giornata n. 1,Sab,Casa,V,2.0,0.0,Monza,...,Andrea Colombo,Report partita,NaN,22.0,172.0,1.0,0,0,2024,Internazionale
1,28-08-2023,20:45,Serie A,Giornata n. 2,Lun,Ospiti,V,2.0,0.0,Cagliari,...,Michael Fabbri,Report partita,NaN,17.0,159.0,0.0,0,0,2024,Internazionale
2,03-09-2023,18:30,Serie A,Giornata n. 3,Dom,Casa,V,4.0,0.0,Fiorentina,...,Matteo Marchetti,Report partita,NaN,20.0,145.0,1.0,1,1,2024,Internazionale
3,16-09-2023,18:00,Serie A,Giornata n. 4,Sab,Casa,V,5.0,1.0,Milan,...,Simone Sozza,Report partita,NaN,13.0,152.0,1.0,1,1,2024,Internazionale
5,24-09-2023,12:30,Serie A,Giornata n. 5,Dom,Ospiti,V,1.0,0.0,Empoli,...,Matteo Marcenaro,Report partita,NaN,23.0,168.0,0.0,0,0,2024,Internazionale


In [4]:
matches.shape #numero di righe e di colonne del dataset

(1360, 26)

### Investighiamo sui dati mancanti

##### Abbiamo un dataset di due stagioni, la seconda stagione è stata calcolata alla 30esima giornata, e 20 squadre, quindi dovremmo avere 30x20 + 38x20

In [5]:
(30*20)+(38*20)

1360

In [6]:
matches["Squadra"].value_counts() 
#numeri di partite per squadra, i valori non sono tutti uguali poichè alcune squadre sono state retrocesse/neopromosse

HellasVerona      69
Internazionale    68
Salernitana       68
Empoli            68
Udinese           68
Lecce             68
Milan             68
Monza             68
Sassuolo          68
Torino            68
Napoli            68
Lazio             68
Roma              68
Bologna           68
Juventus          68
Atalanta          67
Fiorentina        67
Spezia            39
Cremonese         38
Sampdoria         38
Frosinone         30
Cagliari          30
Genoa             30
Name: Squadra, dtype: int64

In [7]:
matches["Girone"].value_counts()
#vediamo quante partite ci sono state per ogni giornata all'interno del dataset

Giornata n. 1              40
Giornata n. 16             40
Giornata n. 30             40
Giornata n. 28             40
Giornata n. 27             40
Giornata n. 21             40
Giornata n. 26             40
Giornata n. 25             40
Giornata n. 24             40
Giornata n. 23             40
Giornata n. 22             40
Giornata n. 2              40
Giornata n. 19             40
Giornata n. 18             40
Giornata n. 17             40
Giornata n. 20             40
Giornata n. 15             40
Giornata n. 8              40
Giornata n. 3              40
Giornata n. 14             40
Giornata n. 5              40
Giornata n. 6              40
Giornata n. 7              40
Giornata n. 4              40
Giornata n. 9              40
Giornata n. 10             40
Giornata n. 11             40
Giornata n. 12             40
Giornata n. 13             40
Giornata n. 29             38
Giornata n. 31             20
Giornata n. 32             20
Giornata n. 33             20
Giornata n

### Pulizia e preparazione dei dati per il machine learning

In [8]:
matches.dtypes

Data               object
Ora                object
Competizione       object
Girone             object
Giorno             object
Stadio             object
Risultato          object
Rf                float64
Rs                float64
Avversario         object
xG                float64
xGA               float64
Poss.             float64
Spettatori        float64
Capitano           object
Formazione         object
Arbitro            object
Report partita     object
Note              float64
Tiri              float64
Dist.             float64
Pun.              float64
Rigori              int64
Rig T               int64
Stagione            int64
Squadra            object
dtype: object

In [9]:
#cambiamo il tipo di dato di "Data", che adesso è object,in datetime
matches["Data"]=pd.to_datetime(matches["Data"])

C:\Users\aless\AppData\Local\Temp\ipykernel_13352\104862989.py:2: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  matches["Data"]=pd.to_datetime(matches["Data"])


In [10]:
matches.dtypes

Data              datetime64[ns]
Ora                       object
Competizione              object
Girone                    object
Giorno                    object
Stadio                    object
Risultato                 object
Rf                       float64
Rs                       float64
Avversario                object
xG                       float64
xGA                      float64
Poss.                    float64
Spettatori               float64
Capitano                  object
Formazione                object
Arbitro                   object
Report partita            object
Note                     float64
Tiri                     float64
Dist.                    float64
Pun.                     float64
Rigori                     int64
Rig T                      int64
Stagione                   int64
Squadra                   object
dtype: object

#### Creiamo dei predittori per il machine learning

##### La prima cosa che faremo è creare un predictor chiamato 'venue_code', che ci servirà per convertire il valore all'interno della colonna Stadio, 'Casa' o 'Ospiti', in una colonna numerica cosi da poter essere letta e usata dall'algoritmo e per poter calcolare il vantaggio del giocare in casa.
##### In questo modo il valore 'Casa' avrà valore 0 e 'Ospiti' valore 1

In [11]:
matches["Venue_code"] = matches["Stadio"].astype("category").cat.codes #cat.codes lo usiamo per convertirlo in integer

In [12]:
matches.head()

,Data,Ora,Competizione,Girone,Giorno,Stadio,Risultato,Rf,Rs,Avversario,...,Report partita,Note,Tiri,Dist.,Pun.,Rigori,Rig T,Stagione,Squadra,Venue_code
0,2023-08-19,20:45,Serie A,Giornata n. 1,Sab,Casa,V,2.0,0.0,Monza,...,Report partita,NaN,22.0,172.0,1.0,0,0,2024,Internazionale,0
1,2023-08-28,20:45,Serie A,Giornata n. 2,Lun,Ospiti,V,2.0,0.0,Cagliari,...,Report partita,NaN,17.0,159.0,0.0,0,0,2024,Internazionale,1
2,2023-03-09,18:30,Serie A,Giornata n. 3,Dom,Casa,V,4.0,0.0,Fiorentina,...,Report partita,NaN,20.0,145.0,1.0,1,1,2024,Internazionale,0
3,2023-09-16,18:00,Serie A,Giornata n. 4,Sab,Casa,V,5.0,1.0,Milan,...,Report partita,NaN,13.0,152.0,1.0,1,1,2024,Internazionale,0
5,2023-09-24,12:30,Serie A,Giornata n. 5,Dom,Ospiti,V,1.0,0.0,Empoli,...,Report partita,NaN,23.0,168.0,0.0,0,0,2024,Internazionale,1


##### Facciamo la stessa cosa per la squadra avversaria, per cui creiamo una variabile 'Opponent_code', cosi che ogni avversario avrà un suo valore numerico associato

In [13]:
matches["opponent_code"]=matches["Avversario"].astype("category").cat.codes

In [14]:
matches.head()

,Data,Ora,Competizione,Girone,Giorno,Stadio,Risultato,Rf,Rs,Avversario,...,Note,Tiri,Dist.,Pun.,Rigori,Rig T,Stagione,Squadra,Venue_code,opponent_code
0,2023-08-19,20:45,Serie A,Giornata n. 1,Sab,Casa,V,2.0,0.0,Monza,...,NaN,22.0,172.0,1.0,0,0,2024,Internazionale,0,14
1,2023-08-28,20:45,Serie A,Giornata n. 2,Lun,Ospiti,V,2.0,0.0,Cagliari,...,NaN,17.0,159.0,0.0,0,0,2024,Internazionale,1,2
2,2023-03-09,18:30,Serie A,Giornata n. 3,Dom,Casa,V,4.0,0.0,Fiorentina,...,NaN,20.0,145.0,1.0,1,1,2024,Internazionale,0,5
3,2023-09-16,18:00,Serie A,Giornata n. 4,Sab,Casa,V,5.0,1.0,Milan,...,NaN,13.0,152.0,1.0,1,1,2024,Internazionale,0,13
5,2023-09-24,12:30,Serie A,Giornata n. 5,Dom,Ospiti,V,1.0,0.0,Empoli,...,NaN,23.0,168.0,0.0,0,0,2024,Internazionale,1,4


In [15]:
#andiamo a sostituire la colonna 'Ora' con la colonna 'hour' che conterrà solo l'ora e non i minuti
# questo valore verrà trattato come un integer da passare all'algoritmo

matches["hour"] = matches["Ora"].str.replace(":.+","", regex=True)

In [16]:
matches.head()

,Data,Ora,Competizione,Girone,Giorno,Stadio,Risultato,Rf,Rs,Avversario,...,Tiri,Dist.,Pun.,Rigori,Rig T,Stagione,Squadra,Venue_code,opponent_code,hour
0,2023-08-19,20:45,Serie A,Giornata n. 1,Sab,Casa,V,2.0,0.0,Monza,...,22.0,172.0,1.0,0,0,2024,Internazionale,0,14,20
1,2023-08-28,20:45,Serie A,Giornata n. 2,Lun,Ospiti,V,2.0,0.0,Cagliari,...,17.0,159.0,0.0,0,0,2024,Internazionale,1,2,20
2,2023-03-09,18:30,Serie A,Giornata n. 3,Dom,Casa,V,4.0,0.0,Fiorentina,...,20.0,145.0,1.0,1,1,2024,Internazionale,0,5,18
3,2023-09-16,18:00,Serie A,Giornata n. 4,Sab,Casa,V,5.0,1.0,Milan,...,13.0,152.0,1.0,1,1,2024,Internazionale,0,13,18
5,2023-09-24,12:30,Serie A,Giornata n. 5,Dom,Ospiti,V,1.0,0.0,Empoli,...,23.0,168.0,0.0,0,0,2024,Internazionale,1,4,12


In [17]:
#l'ultima colonna che creeremo è chiamata 'day_code' che sostituirà i nomi dei giorni della settimana con i valori
# quindi lunedi=0, martedi=1, mercoledi=3, ...
matches["day_code"] = matches["Data"].dt.dayofweek
matches.head()

,Data,Ora,Competizione,Girone,Giorno,Stadio,Risultato,Rf,Rs,Avversario,...,Dist.,Pun.,Rigori,Rig T,Stagione,Squadra,Venue_code,opponent_code,hour,day_code
0,2023-08-19,20:45,Serie A,Giornata n. 1,Sab,Casa,V,2.0,0.0,Monza,...,172.0,1.0,0,0,2024,Internazionale,0,14,20,5
1,2023-08-28,20:45,Serie A,Giornata n. 2,Lun,Ospiti,V,2.0,0.0,Cagliari,...,159.0,0.0,0,0,2024,Internazionale,1,2,20,0
2,2023-03-09,18:30,Serie A,Giornata n. 3,Dom,Casa,V,4.0,0.0,Fiorentina,...,145.0,1.0,1,1,2024,Internazionale,0,5,18,3
3,2023-09-16,18:00,Serie A,Giornata n. 4,Sab,Casa,V,5.0,1.0,Milan,...,152.0,1.0,1,1,2024,Internazionale,0,13,18,5
5,2023-09-24,12:30,Serie A,Giornata n. 5,Dom,Ospiti,V,1.0,0.0,Empoli,...,168.0,0.0,0,0,2024,Internazionale,1,4,12,6


#### Per iniziare abbiamo bisogno di impostare un 'target' che vogliamo provare a predirre
#### Il target che vogliamo ottenere è sapere se la squadra vincerà o no
##### Utilizzeremo la colonna del risultato per leggere se la squadra ha vinto o no (perso o pareggiato) e trasformeremo quel valore in 1 o 0

In [18]:
matches["target"] = (matches["Risultato"] == "V").astype("int")
matches.head()

,Data,Ora,Competizione,Girone,Giorno,Stadio,Risultato,Rf,Rs,Avversario,...,Pun.,Rigori,Rig T,Stagione,Squadra,Venue_code,opponent_code,hour,day_code,target
0,2023-08-19,20:45,Serie A,Giornata n. 1,Sab,Casa,V,2.0,0.0,Monza,...,1.0,0,0,2024,Internazionale,0,14,20,5,1
1,2023-08-28,20:45,Serie A,Giornata n. 2,Lun,Ospiti,V,2.0,0.0,Cagliari,...,0.0,0,0,2024,Internazionale,1,2,20,0,1
2,2023-03-09,18:30,Serie A,Giornata n. 3,Dom,Casa,V,4.0,0.0,Fiorentina,...,1.0,1,1,2024,Internazionale,0,5,18,3,1
3,2023-09-16,18:00,Serie A,Giornata n. 4,Sab,Casa,V,5.0,1.0,Milan,...,1.0,1,1,2024,Internazionale,0,13,18,5,1
5,2023-09-24,12:30,Serie A,Giornata n. 5,Dom,Ospiti,V,1.0,0.0,Empoli,...,0.0,0,0,2024,Internazionale,1,4,12,6,1


## Creaiamo il modello di machine learning
#### Adesso siamo pronti per allenare il nostro modello. 
#### Come modelli utilizzeremo l'algoritmo RandomForest, che è un  modello lineare

In [19]:
from sklearn.ensemble import RandomForestClassifier

In [20]:
rf = RandomForestClassifier(n_estimators=50, min_samples_split=10, random_state=1)

#n_estimators è il numero individuale di decision trees di cui vogliamo fare il train. 
    #più alto sarà il numero degli stimatori, più lungo sarà il tempo impiegato dall'algoritmo ma sarà potenzialmente più accurato
#min_samples_split è il numero di samples che vogliamo avere in una 'foglia' del decision tree
    #più alto è il valore, meno probabile è che ci sia un overfit ma minore sarà la nostra precisione sui dati
    #quindi bisogna fare degli esperimenti su questi valori per trovare quelli ottimali  
#random_state, poichè rf ha tanti parametri random quindi se settiamo un random_state significa che se eseguiamo
    #il rf molte volte otteremo lo stesso risultato, poichè i dati sono gli stessi
    
    

#### Splittiamo i dati in dati di train e di test.
##### Questi dati sono dati temporali quindi dobbiamo fare attenzioni che i dati di test vegnono dopo tutti i dati di training, poichè non si possono usare dati di training nel futuro e utilizzarli per predirre dati del passato.

In [21]:
train = matches[matches["Data"] < '2024-01-01']

In [22]:
test = matches[matches["Data"] > '2024-01-01']

In [23]:
predictors = ["Venue_code","opponent_code", "hour","day_code"]

In [24]:
rf.fit(train[predictors], train["target"])

RandomForestClassifier(min_samples_split=10, n_estimators=50, random_state=1)

In [25]:
# Dopo aver fatto il train, generiamo i predicts, passando i nostri dati di test e i predictors

preds = rf.predict(test[predictors])

#### Adesso abbiamo bisogno di trovare un modo per determinare l'accuratezza del modello.
##### Proveremo un paio di metriche diverse e proviamo a capire quale può avere più senso

In [26]:
from sklearn.metrics import accuracy_score

acc = accuracy_score(test["target"], preds)

In [27]:
acc

0.5966386554621849

In [28]:
combined = pd.DataFrame(dict(actual=test["target"], prediction=preds))

In [29]:
pd.crosstab(index=combined["actual"], columns=combined["prediction"])

prediction,0,1
actual,,
0,121,30
1,66,21


##### Possiamo notare che il nostro modello ha una accuratezza di quasi il 60%. Dalla tabella possiamo notare che quando il risultato è un pareggio o una sconfitta, c'è un'alta probabilità che l'algoritmo indovini, 121 volte ha indovinato e 66 volte ha sbagliato.
###### Quando predictiamo una vittoria invece l'algoritmo sbaglia più delle volte in cui indovina, 30 volte ha sbagliato e 21 ha indovinato.
##### Poichè l'obiettivo è predirre una vittoria, abbiamo bisogno di tornare indietro e revisionare la nostra accuratezza

In [30]:
#utilizziamo un'altra metrica chiamata precisione_score

from sklearn.metrics import precision_score

In [31]:
precision_score(test["target"], preds)

0.4117647058823529

###### Questo risultato conferma l'analisi fatta pocanzi, quando viene prevista una vittoria, la squadra vince solo il 40% delle volte, il che non è un buon risultato e una buona precisione.

## Miglioriamo la precisione con le medie mobili
##### Utilizziamo le medie mobili per avere una accuratezza migliore nell'algoritmo. Per esempio possiamo utilizzare una determinata partita per vedere cosa ha fatto una squadra nelle 3 o 4 partite precedenti e capire in che salute si trovava quella squadra in quel determinato periodo e fare delle previsioni più accurate.

In [32]:
grouped_matches = matches.groupby("Squadra")

In [33]:
group=grouped_matches.get_group("Internazionale")

In [34]:
group

,Data,Ora,Competizione,Girone,Giorno,Stadio,Risultato,Rf,Rs,Avversario,...,Pun.,Rigori,Rig T,Stagione,Squadra,Venue_code,opponent_code,hour,day_code,target
0,2023-08-19,20:45,Serie A,Giornata n. 1,Sab,Casa,V,2.0,0.0,Monza,...,1.0,0,0,2024,Internazionale,0,14,20,5,1
1,2023-08-28,20:45,Serie A,Giornata n. 2,Lun,Ospiti,V,2.0,0.0,Cagliari,...,0.0,0,0,2024,Internazionale,1,2,20,0,1
2,2023-03-09,18:30,Serie A,Giornata n. 3,Dom,Casa,V,4.0,0.0,Fiorentina,...,1.0,1,1,2024,Internazionale,0,5,18,3,1
3,2023-09-16,18:00,Serie A,Giornata n. 4,Sab,Casa,V,5.0,1.0,Milan,...,1.0,1,1,2024,Internazionale,0,13,18,5,1
5,2023-09-24,12:30,Serie A,Giornata n. 5,Dom,Ospiti,V,1.0,0.0,Empoli,...,0.0,0,0,2024,Internazionale,1,4,12,6,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48,2023-06-05,18:00,Serie A,Giornata n. 34,Sab,Ospiti,V,2.0,0.0,Roma,...,0.0,0,0,2023,Internazionale,1,16,18,0,1
50,2023-05-13,20:45,Serie A,Giornata n. 35,Sab,Casa,V,4.0,2.0,Sassuolo,...,0.0,0,0,2023,Internazionale,0,19,20,5,1
52,2023-05-21,18:00,Serie A,Giornata n. 36,Dom,Ospiti,P,1.0,3.0,Napoli,...,0.0,0,0,2023,Internazionale,1,15,18,6,0
54,2023-05-27,20:45,Serie A,Giornata n. 37,Sab,Casa,V,3.0,2.0,Atalanta,...,2.0,0,0,2023,Internazionale,0,0,20,5,1


In [35]:
def rolling_averages(group, cols, new_cols):
    group = group.sort_values("Data")
    rolling_stats = group[cols].rolling(3, closed='left').mean() #prende le 3 partite precedenti e fa la media
    group[new_cols] = rolling_stats
    group = group.dropna(subset=new_cols) #rimuove le righe che non hanno valori, se ad esempio scegliamo una partita che non ha 3 precedenti matches, ad esempio la 2a giornata
    return group
    

In [36]:
cols = ["Rf", "Rs", "Tiri","Dist.","Pun.","Rigori","Rig T"]
new_cols = [f"{c}_rolling" for c in cols]

In [37]:
new_cols

['Rf_rolling',
 'Rs_rolling',
 'Tiri_rolling',
 'Dist._rolling',
 'Pun._rolling',
 'Rigori_rolling',
 'Rig T_rolling']

In [38]:
rolling_averages(group, cols, new_cols)

,Data,Ora,Competizione,Girone,Giorno,Stadio,Risultato,Rf,Rs,Avversario,...,hour,day_code,target,Rf_rolling,Rs_rolling,Tiri_rolling,Dist._rolling,Pun._rolling,Rigori_rolling,Rig T_rolling
11,2022-08-10,15:00,Serie A,Giornata n. 9,Sab,Ospiti,V,2.0,1.0,Sassuolo,...,15,2,1,1.000000,2.333333,15.000000,199.333333,1.000000,0.000000,0.000000
0,2022-08-13,20:45,Serie A,Giornata n. 1,Sab,Ospiti,V,2.0,1.0,Lecce,...,20,5,1,1.333333,2.000000,15.333333,180.666667,0.666667,0.000000,0.000000
1,2022-08-20,20:45,Serie A,Giornata n. 2,Sab,Casa,V,3.0,0.0,Spezia,...,20,5,1,1.333333,1.333333,16.333333,171.666667,0.333333,0.000000,0.000000
2,2022-08-26,20:45,Serie A,Giornata n. 3,Ven,Ospiti,P,1.0,3.0,Lazio,...,20,4,0,2.333333,0.666667,18.000000,160.333333,0.333333,0.000000,0.000000
3,2022-08-30,20:45,Serie A,Giornata n. 4,Mar,Casa,V,3.0,1.0,Cremonese,...,20,1,1,2.000000,1.333333,19.000000,161.666667,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30,2024-04-02,20:45,Serie A,Giornata n. 23,Dom,Casa,V,1.0,0.0,Juventus,...,20,1,1,3.000000,0.333333,14.666667,163.000000,0.333333,0.000000,0.333333
36,2024-04-03,20:45,Serie A,Giornata n. 27,Lun,Casa,V,2.0,1.0,Genoa,...,20,2,1,2.000000,0.333333,16.000000,162.333333,0.333333,0.000000,0.333333
25,2024-06-01,12:30,Serie A,Giornata n. 19,Sab,Casa,V,2.0,1.0,Hellas Verona,...,12,5,1,1.333333,0.666667,16.000000,162.666667,0.333333,0.333333,0.333333
37,2024-09-03,18:00,Serie A,Giornata n. 28,Sab,Ospiti,V,1.0,0.0,Bologna,...,18,1,1,1.666667,0.666667,14.666667,161.666667,0.000000,0.333333,0.333333


In [39]:
matches_rolling = matches.groupby("Squadra").apply(lambda x: rolling_averages(x, cols, new_cols))

In [40]:
matches_rolling

Data    Ora Competizione          Girone Giorno  Stadio  \
Squadra                                                                     
Atalanta 12 2022-05-11  18:00      Serie A  Giornata n. 13    Sab    Casa   
         0  2022-08-13  18:30      Serie A   Giornata n. 1    Sab  Ospiti   
         1  2022-08-21  20:45      Serie A   Giornata n. 2    Dom    Casa   
         2  2022-08-28  18:30      Serie A   Giornata n. 3    Dom  Ospiti   
         8  2022-09-10  15:00      Serie A   Giornata n. 9    Dom  Ospiti   
...                ...    ...          ...             ...    ...     ...   
Udinese  24 2024-03-02  15:00      Serie A  Giornata n. 23    Sab    Casa   
         30 2024-03-16  15:00      Serie A  Giornata n. 29    Sab    Casa   
         20 2024-07-01  15:00      Serie A  Giornata n. 19    Dom    Casa   
         29 2024-11-03  20:45      Serie A  Giornata n. 28    Lun  Ospiti   
         25 2024-12-02  20:45      Serie A  Giornata n. 24    Lun  Ospiti   

            Risultato   Rf   Rs     Avversario  ...  hour  day_code  target  \
Squadra                                         ...                           
Atalanta 12         P  1.0  2.0         Napoli  ...    18         2       0   
         0          V  2.0  0.0      Sampdoria  ...    18         5       1   
         1          N  1.0  1.0          Milan  ...    20         6       0   
         2          V  1.0  0.0  Hellas Verona  ...    18         6       1   
         8          N  2.0  2.0        Udinese  ...    15         5       0   
...               ...  ...  ...            ...  ...   ...       ...     ...   
Udinese  24         N  0.0  0.0          Monza  ...    15         5       0   
         30         P  0.0  2.0         Torino  ...    15         5       0   
         20         P  1.0  2.0          Lazio  ...    15         0       0   
         29         V  2.0  1.0          Lazio  ...    20         6       1   
         25         V  1.0  0.0       Juventus  ...    20         0       1   

             Rf_rolling Rs_rolling Tiri_rolling Dist._rolling Pun._rolling  \
Squadra                                                                      
Atalanta 12    2.000000   0.333333    11.000000    165.000000     0.333333   
         0     1.333333   0.666667    12.666667    167.333333     0.333333   
         1     1.666667   0.666667    12.333333    163.666667     0.000000   
         2     1.333333   1.000000    11.666667    176.000000     0.000000   
         8     1.333333   0.333333    11.666667    174.000000     0.000000   
...                 ...        ...          ...           ...          ...   
Udinese  24    0.666667   1.333333    13.333333    166.333333     0.666667   
         30    0.333333   1.000000    14.333333    184.666667     0.333333   
         20    0.000000   1.333333    12.666667    194.000000     0.333333   
         29    0.333333   1.333333    15.333333    208.333333     0.666667   
         25    1.000000   1.666667    13.000000    215.666667     0.666667   

             Rigori_rolling  Rig T_rolling  
Squadra                                     
Atalanta 12        0.666667       0.666667  
         0         0.333333       0.333333  
         1         0.333333       0.333333  
         2         0.333333       0.333333  
         8         0.000000       0.000000  
...                     ...            ...  
Udinese  24        0.000000       0.000000  
         30        0.000000       0.000000  
         20        0.000000       0.000000  
         29        0.000000       0.000000  
         25        0.000000       0.000000  

[1291 rows x 38 columns]

In [41]:
matches_rolling = matches_rolling.droplevel("Squadra")

In [42]:
matches_rolling.head()

,Data,Ora,Competizione,Girone,Giorno,Stadio,Risultato,Rf,Rs,Avversario,...,hour,day_code,target,Rf_rolling,Rs_rolling,Tiri_rolling,Dist._rolling,Pun._rolling,Rigori_rolling,Rig T_rolling
12,2022-05-11,18:00,Serie A,Giornata n. 13,Sab,Casa,P,1.0,2.0,Napoli,...,18,2,0,2.000000,0.333333,11.000000,165.000000,0.333333,0.666667,0.666667
0,2022-08-13,18:30,Serie A,Giornata n. 1,Sab,Ospiti,V,2.0,0.0,Sampdoria,...,18,5,1,1.333333,0.666667,12.666667,167.333333,0.333333,0.333333,0.333333
1,2022-08-21,20:45,Serie A,Giornata n. 2,Dom,Casa,N,1.0,1.0,Milan,...,20,6,0,1.666667,0.666667,12.333333,163.666667,0.000000,0.333333,0.333333
2,2022-08-28,18:30,Serie A,Giornata n. 3,Dom,Ospiti,V,1.0,0.0,Hellas Verona,...,18,6,1,1.333333,1.000000,11.666667,176.000000,0.000000,0.333333,0.333333
8,2022-09-10,15:00,Serie A,Giornata n. 9,Dom,Ospiti,N,2.0,2.0,Udinese,...,15,5,0,1.333333,0.333333,11.666667,174.000000,0.000000,0.000000,0.000000


In [43]:
matches_rolling.index = range(matches_rolling.shape[0])

In [44]:
matches_rolling

,Data,Ora,Competizione,Girone,Giorno,Stadio,Risultato,Rf,Rs,Avversario,...,hour,day_code,target,Rf_rolling,Rs_rolling,Tiri_rolling,Dist._rolling,Pun._rolling,Rigori_rolling,Rig T_rolling
0,2022-05-11,18:00,Serie A,Giornata n. 13,Sab,Casa,P,1.0,2.0,Napoli,...,18,2,0,2.000000,0.333333,11.000000,165.000000,0.333333,0.666667,0.666667
1,2022-08-13,18:30,Serie A,Giornata n. 1,Sab,Ospiti,V,2.0,0.0,Sampdoria,...,18,5,1,1.333333,0.666667,12.666667,167.333333,0.333333,0.333333,0.333333
2,2022-08-21,20:45,Serie A,Giornata n. 2,Dom,Casa,N,1.0,1.0,Milan,...,20,6,0,1.666667,0.666667,12.333333,163.666667,0.000000,0.333333,0.333333
3,2022-08-28,18:30,Serie A,Giornata n. 3,Dom,Ospiti,V,1.0,0.0,Hellas Verona,...,18,6,1,1.333333,1.000000,11.666667,176.000000,0.000000,0.333333,0.333333
4,2022-09-10,15:00,Serie A,Giornata n. 9,Dom,Ospiti,N,2.0,2.0,Udinese,...,15,5,0,1.333333,0.333333,11.666667,174.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1286,2024-03-02,15:00,Serie A,Giornata n. 23,Sab,Casa,N,0.0,0.0,Monza,...,15,5,0,0.666667,1.333333,13.333333,166.333333,0.666667,0.000000,0.000000
1287,2024-03-16,15:00,Serie A,Giornata n. 29,Sab,Casa,P,0.0,2.0,Torino,...,15,5,0,0.333333,1.000000,14.333333,184.666667,0.333333,0.000000,0.000000
1288,2024-07-01,15:00,Serie A,Giornata n. 19,Dom,Casa,P,1.0,2.0,Lazio,...,15,0,0,0.000000,1.333333,12.666667,194.000000,0.333333,0.000000,0.000000
1289,2024-11-03,20:45,Serie A,Giornata n. 28,Lun,Ospiti,V,2.0,1.0,Lazio,...,20,6,1,0.333333,1.333333,15.333333,208.333333,0.666667,0.000000,0.000000


## RETRAINING DEL MODELLO

In [45]:
def make_predictions(data, predictors):
    train = data[data["Data"] < '2024-01-01']
    test = data[data["Data"] > '2024-01-01']
    rf.fit(train[predictors], train["target"])
    preds = rf.predict(test[predictors])
    combined = pd.DataFrame(dict(actual=test["target"], prediction=preds), index=test.index)
    precision = precision_score(test["target"], preds)
    return combined, precision

In [46]:
combined, precision = make_predictions(matches_rolling, predictors + new_cols)

In [47]:
precision

0.46153846153846156

###### Abbiamo migliorato di un po' la nostra precisione, passando da 41% a 46%

In [48]:
combined

,actual,prediction
53,1,1
54,1,0
55,1,1
56,0,1
57,0,0
...,...,...
1286,0,0
1287,0,0
1288,0,0
1289,1,0


In [49]:
combined = combined.merge(matches_rolling[["Data","Squadra","Avversario","Risultato"]], left_index=True, right_index=True)

In [50]:
combined

,actual,prediction,Data,Squadra,Avversario,Risultato
53,1,1,2024-01-15,Atalanta,Frosinone,V
54,1,0,2024-01-27,Atalanta,Udinese,V
55,1,1,2024-02-17,Atalanta,Sassuolo,V
56,0,1,2024-02-25,Atalanta,Milan,N
57,0,0,2024-02-28,Atalanta,Inter,P
...,...,...,...,...,...,...
1286,0,0,2024-03-02,Udinese,Monza,N
1287,0,0,2024-03-16,Udinese,Torino,P
1288,0,0,2024-07-01,Udinese,Lazio,P
1289,1,0,2024-11-03,Udinese,Lazio,V


##### Un ultima cosa da fare è combinare i nomi diversi che sono date alle stesse squadre nel dataset, come ad esempio Inter e Internazione o Hellas Verona e HellasVerona

In [51]:
class MissingDict(dict):
    __missing__ = lambda self, key:key
    
map_values = {
    "Internazionale": "Inter",
    "HellasVerona": "Hellas Verona"
}
mapping=MissingDict(**map_values)

In [52]:
mapping["Internazionale"]

'Inter'

In [53]:
combined["new_Squadra"] = combined["Squadra"].map(mapping)

In [54]:
combined

,actual,prediction,Data,Squadra,Avversario,Risultato,new_Squadra
53,1,1,2024-01-15,Atalanta,Frosinone,V,Atalanta
54,1,0,2024-01-27,Atalanta,Udinese,V,Atalanta
55,1,1,2024-02-17,Atalanta,Sassuolo,V,Atalanta
56,0,1,2024-02-25,Atalanta,Milan,N,Atalanta
57,0,0,2024-02-28,Atalanta,Inter,P,Atalanta
...,...,...,...,...,...,...,...
1286,0,0,2024-03-02,Udinese,Monza,N,Udinese
1287,0,0,2024-03-16,Udinese,Torino,P,Udinese
1288,0,0,2024-07-01,Udinese,Lazio,P,Udinese
1289,1,0,2024-11-03,Udinese,Lazio,V,Udinese


In [56]:
merged = combined.merge(combined, left_on=["Data","new_Squadra"], right_on=["Data","Avversario"])

In [57]:
merged

,actual_x,prediction_x,Data,Squadra_x,Avversario_x,Risultato_x,new_Squadra_x,actual_y,prediction_y,Squadra_y,Avversario_y,Risultato_y,new_Squadra_y
0,1,1,2024-01-15,Atalanta,Frosinone,V,Atalanta,0,0,Frosinone,Atalanta,P,Frosinone
1,1,0,2024-01-27,Atalanta,Udinese,V,Atalanta,0,0,Udinese,Atalanta,P,Udinese
2,1,1,2024-02-17,Atalanta,Sassuolo,V,Atalanta,0,0,Sassuolo,Atalanta,P,Sassuolo
3,0,1,2024-02-25,Atalanta,Milan,N,Atalanta,0,1,Milan,Atalanta,N,Milan
4,0,0,2024-02-28,Atalanta,Inter,P,Atalanta,1,0,Internazionale,Atalanta,V,Inter
...,...,...,...,...,...,...,...,...,...,...,...,...,...
233,0,0,2024-03-02,Udinese,Monza,N,Udinese,0,0,Monza,Udinese,N,Monza
234,0,0,2024-03-16,Udinese,Torino,P,Udinese,1,0,Torino,Udinese,V,Torino
235,0,0,2024-07-01,Udinese,Lazio,P,Udinese,1,0,Lazio,Udinese,V,Lazio
236,1,0,2024-11-03,Udinese,Lazio,V,Udinese,0,0,Lazio,Udinese,P,Lazio


In [59]:
merged[(merged["prediction_x"]==1) & (merged["prediction_y"]==0)]["actual_x"].value_counts()

1    17
0    16
Name: actual_x, dtype: int64

In [61]:
17/33

0.5151515151515151

##### Un altro modo per calcolare e migliorare l'accuratezza. In questo modo siamo riusciti a migliorare la nostra percentuale fino a 51%